In [3]:
from pyspark.sql import SparkSession
from ibm_botocore.client import Config
import ibm_boto3

In [5]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL") \
    .getOrCreate()

In [6]:
# ARQUIVOS SALVOS EM UM BUCKET GRATUITO DA IBM
# CREDENCIAIS DO BUCKET

credentials_7 = {
    'IAM_SERVICE_ID': 'iam-ServiceId-9ec669c5-6c1d-4d70-810c-1941badb231e',
    'IBM_API_KEY_ID': 'Ta-EpXfATe3TsngSYxlAdhNCO5n_zrlh5GL7x7-XOauv',
    'ENDPOINT': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'IBM_AUTH_ENDPOINT': 'https://iam.ng.bluemix.net/oidc/token',
    'BUCKET': 'semantixspark-donotdelete-pr-uclrrmcdckxuwn',
    'FILE': 'access_log_Jul95'
}

credentials_8 = {
    'IAM_SERVICE_ID': 'iam-ServiceId-9ec669c5-6c1d-4d70-810c-1941badb231e',
    'IBM_API_KEY_ID': 'Ta-EpXfATe3TsngSYxlAdhNCO5n_zrlh5GL7x7-XOauv',
    'ENDPOINT': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'IBM_AUTH_ENDPOINT': 'https://iam.ng.bluemix.net/oidc/token',
    'BUCKET': 'semantixspark-donotdelete-pr-uclrrmcdckxuwn',
    'FILE': 'access_log_Aug95'
}

In [7]:
# CLIENT PARA DOWNLOAD DOS ARQUIVOS

# JULHO
cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials_7['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials_7['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials_7['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials_7['ENDPOINT'])

cos.download_file(Bucket=credentials_7['BUCKET']
                  ,Key='access_log_Jul95'
                  ,Filename='access_log_Jul95')

# AGOSTO
cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials_8['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials_8['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials_8['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials_8['ENDPOINT'])

cos.download_file(Bucket=credentials_8['BUCKET']
                  ,Key='access_log_Aug95'
                  ,Filename='access_log_Aug95')


In [9]:
# lEITURA COM O SPARK

files = ['access_log_Jul95','access_log_Aug95']

df1 = spark.read.load(files
                     , format='csv'
                     , sep=' '
                     , inferSchema='true'
                     , header='false'
                    )

df2 = df1.createOrReplaceTempView("NASA")

sqlDF = spark.sql("SELECT * FROM NASA")
sqlDF.show()

+--------------------+---+---+--------------------+------+--------------------+---+-----+
|                 _c0|_c1|_c2|                 _c3|   _c4|                 _c5|_c6|  _c7|
+--------------------+---+---+--------------------+------+--------------------+---+-----+
|        199.72.81.55|  -|  -|[01/Jul/1995:00:0...|-0400]|GET /history/apol...|200| 6245|
|unicomp6.unicomp.net|  -|  -|[01/Jul/1995:00:0...|-0400]|GET /shuttle/coun...|200| 3985|
|      199.120.110.21|  -|  -|[01/Jul/1995:00:0...|-0400]|GET /shuttle/miss...|200| 4085|
|  burger.letters.com|  -|  -|[01/Jul/1995:00:0...|-0400]|GET /shuttle/coun...|304|    0|
|      199.120.110.21|  -|  -|[01/Jul/1995:00:0...|-0400]|GET /shuttle/miss...|200| 4179|
|  burger.letters.com|  -|  -|[01/Jul/1995:00:0...|-0400]|GET /images/NASA-...|304|    0|
|  burger.letters.com|  -|  -|[01/Jul/1995:00:0...|-0400]|GET /shuttle/coun...|200|    0|
|     205.212.115.106|  -|  -|[01/Jul/1995:00:0...|-0400]|GET /shuttle/coun...|200| 3985|
|         

In [83]:
query = ''' 
        CREATE OR REPLACE TEMPORARY VIEW FATO_NASA
        AS
        SELECT _c0 as host
            , FROM_UNIXTIME(UNIX_TIMESTAMP(SUBSTRING( concat( _c3 , _c4 ) , 2 , 20), 'dd/MMM/yyyy:HH:mm:ss'),'yyyy-MM-dd HH:mm:ss') as timestamp
            , _c5 as requisicao
            , _c6 as retorno
            , _c7 as bytes
        FROM NASA
        '''
sqlDF = spark.sql(query)

In [84]:
sqlDF = spark.sql('SELECT * FROM FATO_NASA')
sqlDF.show(truncate=False)

+-------------------------+-------------------+--------------------------------------------------------------+-------+-----+
|host                     |timestamp          |requisicao                                                    |retorno|bytes|
+-------------------------+-------------------+--------------------------------------------------------------+-------+-----+
|199.72.81.55             |1995-07-01 00:00:01|GET /history/apollo/ HTTP/1.0                                 |200    |6245 |
|unicomp6.unicomp.net     |1995-07-01 00:00:06|GET /shuttle/countdown/ HTTP/1.0                              |200    |3985 |
|199.120.110.21           |1995-07-01 00:00:09|GET /shuttle/missions/sts-73/mission-sts-73.html HTTP/1.0     |200    |4085 |
|burger.letters.com       |1995-07-01 00:00:11|GET /shuttle/countdown/liftoff.html HTTP/1.0                  |304    |0    |
|199.120.110.21           |1995-07-01 00:00:11|GET /shuttle/missions/sts-73/sts-73-patch-small.gif HTTP/1.0  |200    |4179 |


In [22]:
sqlDF = spark.sql('SELECT COUNT(DISTINCT HOST) AS HOSTS_UNICOS FROM FATO_NASA')
sqlDF.show()

+------------+
|HOSTS_UNICOS|
+------------+
|      137979|
+------------+



In [24]:
query = ''' 
        SELECT COUNT(1) AS TOTAL_ERROS_404
        FROM FATO_NASA
        WHERE RETORNO = '404'
        '''

sqlDF = spark.sql(query)
sqlDF.show()

+---------------+
|TOTAL_ERROS_404|
+---------------+
|          20871|
+---------------+



In [34]:
query = ''' 
        SELECT REQUISICAO
            , COUNT(1) AS ERROS404
        FROM FATO_NASA
        WHERE RETORNO = '404'
        GROUP BY REQUISICAO
        ORDER BY COUNT(1) DESC
        LIMIT 5
        '''
sqlDF = spark.sql(query)
sqlDF.show(truncate=False)

+---------------------------------------------------------+---------------+
|REQUISICAO                                               |TOTAL_ERROS_404|
+---------------------------------------------------------+---------------+
|GET /pub/winvn/readme.txt HTTP/1.0                       |2004           |
|GET /pub/winvn/release.txt HTTP/1.0                      |1732           |
|GET /shuttle/missions/STS-69/mission-STS-69.html HTTP/1.0|682            |
|GET /shuttle/missions/sts-68/ksc-upclose.gif HTTP/1.0    |426            |
|GET /history/apollo/a-001/a-001-patch-small.gif HTTP/1.0 |384            |
+---------------------------------------------------------+---------------+



In [91]:
query = ''' 
        SELECT to_date(TIMESTAMP) AS DATA
            , COUNT(1) AS ERROS404
        FROM FATO_NASA
        WHERE RETORNO = '404'
        GROUP BY to_date(TIMESTAMP)
        ORDER BY 1 ASC
        '''
sqlDF = spark.sql(query)
sqlDF.show(100, truncate=False)

+----------+--------+
|DATA      |ERROS404|
+----------+--------+
|1995-07-01|316     |
|1995-07-02|291     |
|1995-07-03|470     |
|1995-07-04|359     |
|1995-07-05|497     |
|1995-07-06|640     |
|1995-07-07|569     |
|1995-07-08|302     |
|1995-07-09|348     |
|1995-07-10|398     |
|1995-07-11|471     |
|1995-07-12|470     |
|1995-07-13|531     |
|1995-07-14|411     |
|1995-07-15|254     |
|1995-07-16|257     |
|1995-07-17|406     |
|1995-07-18|465     |
|1995-07-19|638     |
|1995-07-20|428     |
|1995-07-21|332     |
|1995-07-22|191     |
|1995-07-23|233     |
|1995-07-24|328     |
|1995-07-25|461     |
|1995-07-26|336     |
|1995-07-27|336     |
|1995-07-28|94      |
|1995-08-01|243     |
|1995-08-03|303     |
|1995-08-04|346     |
|1995-08-05|236     |
|1995-08-06|373     |
|1995-08-07|537     |
|1995-08-08|390     |
|1995-08-09|279     |
|1995-08-10|306     |
|1995-08-11|263     |
|1995-08-12|196     |
|1995-08-13|216     |
|1995-08-14|287     |
|1995-08-15|327     |
|1995-08-1

In [96]:
query = ''' 
        SELECT SUM(BYTES)*1.0/(1024*1024*1024) as GIGABYTES
        FROM FATO_NASA
        '''
sqlDF = spark.sql(query)
sqlDF.show(truncate=False)

+------------------+
|GIGABYTES         |
+------------------+
|61.024278212338686|
+------------------+

